# 🗺️ Zoom_to_adress: R Notebook
Dieses Notebook lädt automatisch alle benötigten Pakete, definiert die Funktion `meine_funktion()` und führt sie online aus (z. B. auf Binder oder JupyterLab).

In [ ]:
# -------------------------
# 📦 Pakete prüfen & laden
# -------------------------
for (pkg in c('tidygeocoder', 'dplyr', 'terra', 'stringr', 'raster', 'curl')) {
  if (!requireNamespace(pkg, quietly = TRUE)) install.packages(pkg)
  library(pkg, character.only = TRUE)
}

In [ ]:
# -------------------------
# 🧭 FUNKTION: meine_funktion()
# -------------------------
meine_funktion <- function(p_adresse, p_save_path, p_buffer_1, p_buffer_2){
  
  # Pakete
  library(tidygeocoder)
  library(dplyr)
  library(terra)
  library(raster)
  
  # Adresse geokodieren
  l_adresse <- p_adresse %>% geocode(address = adresse, method = 'osm')
  l_adresse <- data.frame(l_adresse)
  v <- vect(l_adresse, geom = c('long', 'lat'), crs = 'EPSG:4326')
  v_utm <- project(v, 'EPSG:25832')
  v_gk3 <- project(v, 'EPSG:31467')

  # Pfade vorbereiten
  p_tif <- data.frame(x = as.numeric(substr(geom(v_utm)[,'x'], 1 , 3)), y = as.numeric(substr(geom(v_utm)[,'y'], 1 , 4)))
  l_url <- paste('https://opengeodata.lgl-bw.de/data/dgm/', 'dgm1_32_', p_tif[1, 'x'], '_', p_tif[1, 'y'], '_2_bw', '.zip', sep = '')
  destfile <- paste0(p_save_path, '/', p_tif[, 'x'], '_', p_tif[, 'y'], '_2_bw.zip')

  # Download
  download.file(l_url, destfile = destfile, method = 'curl')
  unzip(destfile, exdir = p_save_path)

  # Dateien lesen
  unzipped_folder <- paste0(p_save_path, '/dgm1_32_', p_tif[1,'x'], '_', p_tif[1,'y'], '_2_bw')
  p_xyz <- list.files(unzipped_folder, pattern = '\\.xyz$', full.names = TRUE)
  if (length(p_xyz) == 0) stop('Keine XYZ-Dateien gefunden.')

  list_xyz <- lapply(p_xyz, function(f) read.csv(f, sep = '', dec = ',', header = FALSE))
  dgm_df_all <- do.call(rbind, list_xyz)
  dgm <- rasterFromXYZ(dgm_df_all, crs='+init=EPSG:25832')
  dgm <- projectRaster(dgm, crs='+init=epsg:31467')
  dgm <- rast(dgm)

  # Plotfunktion
  zoom_to_point <- function(raster, point, buffer = 200, col_point = 'red') {
    pt <- geom(point)
    x <- pt[1, 'x']; y <- pt[1, 'y']
    xlim <- c(x - buffer, x + buffer)
    ylim <- c(y - buffer, y + buffer)
    plot(raster, xlim = xlim, ylim = ylim)
    plot(point, add = TRUE, pch = 16, col = col_point, cex = 2)
  }

  # Plotten
  par(mfrow = c(1, 3))
  plot(dgm)
  plot(v_gk3, add = TRUE, pch = 16, col = 'red')
  zoom_to_point(dgm, v_gk3, buffer = p_buffer_1)
  zoom_to_point(dgm, v_gk3, buffer = p_buffer_2)

  message('✅ Fertig! Alles durchgelaufen.')
  return(list(raster = dgm, punkt = v_gk3))
}

In [ ]:
# -------------------------
# 🚀 Aufruf der Funktion
# -------------------------
meine_funktion(
  p_adresse = data.frame(adresse = 'Belfortstraße 15, 79098 Freiburg, Deutschland'),
  p_save_path = '/home/jovyan',
  p_buffer_1 = 100,
  p_buffer_2 = 10
)